In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
dataDir = microsimDir+"/NOTEBOOKS/DATA"

#contains person-year information from a Microsim NHANES simulation
df = pd.read_csv(dataDir+"/nhanes-normality-test-4.csv")

In [2]:
#df.columns.to_list()

In [3]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [4]:
#for col in categoricalVars:
#    print(df[col].value_counts())

In [5]:
%%time
nCategories = 0
sizesList = list()
for gender in set(df["gender"].tolist()):
    for smoking in set(df["smokingStatus"].tolist()):
        print(gender,smoking)
        for raceEthnicity in set(df["raceEthnicity"].tolist()):
            for statin in set(df["statin"].tolist()):
                for education in set(df["education"].tolist()):
                    for alcoholPerWeek in set(df["alcoholPerWeek"].tolist()):
                        for anyPhysicalActivity in set(df["anyPhysicalActivity"].tolist()):
                            for antiHypertensiveCount in set(df["antiHypertensiveCount"].tolist()):
                                
                                nCategories += 1
                                
                                dfForGroup = df.loc[(df["gender"]==gender) & 
                                                        (df["smokingStatus"]==smoking) &
                                                        (df["raceEthnicity"]==raceEthnicity) &
                                                        (df["statin"]==statin) &
                                                        (df["education"]==education) &
                                                        (df["alcoholPerWeek"]==alcoholPerWeek) &
                                                        (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                                        (df["antiHypertensiveCount"]==antiHypertensiveCount), :].copy()
                                
                                size = dfForGroup.shape[0]
                                
                                #data = np.array(df.loc[(df["gender"]==gender) & 
                                #                        (df["smokingStatus"]==smoking) &
                                #                        (df["raceEthnicity"]==raceEthnicity) &
                                #                        (df["statin"]==statin) &
                                #                        (df["education"]==education) &
                                #                        (df["alcoholPerWeek"]==alcoholPerWeek) &
                                #                        (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                #                        (df["antiHypertensiveCount"]==antiHypertensiveCount), 
                                #                       continuousVars].copy())
                                
                                #size = data.shape[0]
                                
                                if size>0:
                                    group = [gender, smoking, raceEthnicity, statin, education, alcoholPerWeek, 
                                         anyPhysicalActivity, antiHypertensiveCount]
                                    names = dfForGroup["name"].tolist()
                                    sizesList += [[group, size, names]]
                                                    

1 0
1 1
1 2
2 0
2 1
2 2
CPU times: user 5.86 s, sys: 3.92 ms, total: 5.86 s
Wall time: 5.87 s


In [6]:
groupsList = sizesList
totalPeople = sum(list(map( lambda x: x[1], sizesList)))

In [7]:
with open(dataDir+"/nhanesMeans-4.csv", "w") as meansFile, open(dataDir+"/nhanesCovs-4.csv","w") as covsFile:
    pd.DataFrame(categoricalVars+["weight"]+continuousVars).T.to_csv(meansFile, header=True, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(covsFile, header=True, index=False)
    for i,groupWithSize in enumerate(groupsList):
        if i%500==0:
            print(i)
        group = groupWithSize[0]
        size = groupWithSize[1]
    
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
    
        data = np.array(df.loc[(df["gender"]==gender) & 
                                (df["smokingStatus"]==smoking) &
                                (df["raceEthnicity"]==raceEthnicity) &
                                (df["statin"]==statin) &
                                (df["education"]==education) &
                                (df["alcoholPerWeek"]==alcoholPerWeek) &
                                (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                (df["antiHypertensiveCount"]==antiHypertensiveCount), 
                                                        continuousVars].copy())
    
        distMean, distCov = multivariate_normal.fit(data)
        
        #1 row per block because means are of shape (1,4)
        distMeanDf = pd.DataFrame(distMean)
        pd.concat(
                    [ pd.DataFrame([gender, smoking, raceEthnicity, statin, 
                                    education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount, size/totalPeople]).T,
                      distMeanDf.T],axis=1, ignore_index=True).to_csv(meansFile, header=False, index=False)
        meansFile.write("\n\n")
                    
        #4 rows per block because covs are of shape (4,4)
        distCovDf = pd.DataFrame(distCov)
        pd.concat(
                [ pd.concat([pd.DataFrame([gender, smoking, raceEthnicity, statin,
                                           education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount]).T]*len(continuousVars),ignore_index=True),
                  distCovDf.T],axis=1, ignore_index=True).to_csv(covsFile, header=False, index=False)
        covsFile.write("\n\n")

0
500
1000
1500


In [8]:
meansDf = pd.read_csv(dataDir+"/nhanesMeans-4.csv", header=1)
covsDf = pd.read_csv(dataDir+"/nhanesCovs-4.csv", header=1)

In [9]:
covsDf.head()

,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,1,0,1,False,1,0,0,0.0,463.876543,72.543210,20.238272,753.456790,69.098765,17.362963,576.580247,94.853086,0.946914,409.851852,-24.962963
1,1,0,1,False,1,0,0,0.0,72.543210,38.765432,3.784938,-12.765432,82.876543,-1.359259,-58.197531,11.041975,-0.491975,57.037037,20.962963
2,1,0,1,False,1,0,0,0.0,20.238272,3.784938,7.346980,64.472840,77.884938,1.681704,41.589012,28.761543,0.063568,43.659259,6.572593
3,1,0,1,False,1,0,0,0.0,753.456790,-12.765432,64.472840,3549.876543,1461.901235,55.025926,2961.975309,255.091358,4.398642,668.666667,-135.037037
4,1,0,1,False,1,0,0,0.0,69.098765,82.876543,77.884938,1461.901235,3714.098765,-17.137037,939.358025,176.519753,-3.834198,87.481481,562.814815


In [10]:
for groupWithSize in groupsList:
    group = groupWithSize[0]
    size = groupWithSize[1]
    
    gender = group[0]
    smoking = group[1]
    raceEthnicity = group[2]
    statin = group[3]
    education = group[4]
    alcoholPerWeek = group[5]
    anyPhysicalActivity = group[6] 
    antiHypertensiveCount = group[7]
    
    gCov = covsDf.loc[ (covsDf["gender"]==gender) &
             (covsDf["smokingStatus"]==smoking) &
             (covsDf["raceEthnicity"]==raceEthnicity) &
             (covsDf["statin"]==statin) &
             (covsDf["education"]==education) &
             (covsDf["alcoholPerWeek"]==alcoholPerWeek) &
             (covsDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (covsDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    #if np.linalg.matrix_rank(gCov) < gCov.shape[0]:
    #    groupWithSize += [True]
    #else:
    #    groupWithSize += [False]
        
    #if (np.linalg.matrix_rank(gCov) < gCov.shape[0]) | (not np.all(np.linalg.eig(gCov)[0]>0)) | (not np.allclose(gCov, gCov.T)):
    if not np.all(np.linalg.eig(gCov)[0]>10**(-3)):
        groupWithSize += [True]
    else:
        groupWithSize += [False]

In [11]:
sum(list(map(lambda x: x[3], groupsList)))

1531

In [12]:
for groupWithSize in groupsList:
    group = groupWithSize[0]
    size = groupWithSize[1]
    singular = groupWithSize[3]
    
    if singular:
        #print(group)
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
        
        gMean = meansDf.loc[ (meansDf["gender"]==gender) &
             (meansDf["smokingStatus"]==smoking) &
             (meansDf["raceEthnicity"]==raceEthnicity) &
             (meansDf["statin"]==statin) &
             (meansDf["education"]==education) &
             (meansDf["alcoholPerWeek"]==alcoholPerWeek) &
             (meansDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (meansDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
        
        altGroupsList = list()
        for altGroupWithSize in groupsList:
            altGroup = altGroupWithSize[0]
            altSize = altGroupWithSize[1]
            altSingular = altGroupWithSize[3]
            
            if not altSingular:
                altGender = altGroup[0]
                altSmoking = altGroup[1]
                altRaceEthnicity = altGroup[2]
                altStatin = altGroup[3]
                altEducation = altGroup[4]
                altAlcoholPerWeek = altGroup[5]
                altAnyPhysicalActivity = altGroup[6] 
                altAntiHypertensiveCount = altGroup[7]
    
                altGMean = meansDf.loc[ (meansDf["gender"]==altGender) &
                                     (meansDf["smokingStatus"]==altSmoking) &
                                     (meansDf["raceEthnicity"]==altRaceEthnicity) &
                                     (meansDf["statin"]==altStatin) &
                                     (meansDf["education"]==altEducation) &
                                     (meansDf["alcoholPerWeek"]==altAlcoholPerWeek) &
                                     (meansDf["anyPhysicalActivity"]==altAnyPhysicalActivity) &
                                     (meansDf["antiHypertensiveCount"]==altAntiHypertensiveCount), continuousVars]
    
                altGCov = covsDf.loc[ (covsDf["gender"]==altGender) &
                                   (covsDf["smokingStatus"]==altSmoking) &
                                   (covsDf["raceEthnicity"]==altRaceEthnicity) &
                                   (covsDf["statin"]==altStatin) &
                                   (covsDf["education"]==altEducation) &
                                   (covsDf["alcoholPerWeek"]==altAlcoholPerWeek) &
                                   (covsDf["anyPhysicalActivity"]==altAnyPhysicalActivity) &
                                   (covsDf["antiHypertensiveCount"]==altAntiHypertensiveCount), continuousVars]
    
                gDist = multivariate_normal(np.array(altGMean)[0], np.array(altGCov), allow_singular=False)
        
                altGroupsList += [[altGroup, gDist.pdf(gMean)]] #[altGroup, gDist.pdf(group)]
    
        probabilities = list(map(lambda x: x[1], altGroupsList))
        maxProb = max(probabilities)
        altGroupWithMaxProb = altGroupsList[probabilities.index(maxProb)][0]
        groupWithSize += [altGroupWithMaxProb]
    else:
        groupWithSize += [None]

In [13]:
groupsList[0]

[[1, 0, 1, False, 1, 0, 0, 0.0],
 9,
 [41909, 42688, 43025, 43390, 44501, 44548, 45255, 45756, 45914],
 True,
 [1, 2, 1, False, 1, 3, 1, 0.0]]

In [14]:
personsDf = pd.DataFrame(data=None, columns= ["name"]+categoricalVars+continuousVars)
#popSize = 100
for groupWithSize in groupsList:
    group = groupWithSize[0]
    size = groupWithSize[1]
    names = groupWithSize[2]
    singular = groupWithSize[3]
    altGroup = groupWithSize[4]
    
    if singular:
        gender = altGroup[0]
        smoking = altGroup[1]
        raceEthnicity = altGroup[2]
        statin = altGroup[3]
        education = altGroup[4]
        alcoholPerWeek = altGroup[5]
        anyPhysicalActivity = altGroup[6] 
        antiHypertensiveCount = altGroup[7]
    else:
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
    
    gMean = meansDf.loc[ (meansDf["gender"]==gender) &
             (meansDf["smokingStatus"]==smoking) &
             (meansDf["raceEthnicity"]==raceEthnicity) &
             (meansDf["statin"]==statin) &
             (meansDf["education"]==education) &
             (meansDf["alcoholPerWeek"]==alcoholPerWeek) &
             (meansDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (meansDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    gCov = covsDf.loc[ (covsDf["gender"]==gender) &
             (covsDf["smokingStatus"]==smoking) &
             (covsDf["raceEthnicity"]==raceEthnicity) &
             (covsDf["statin"]==statin) &
             (covsDf["education"]==education) &
             (covsDf["alcoholPerWeek"]==alcoholPerWeek) &
             (covsDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (covsDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    gDist = multivariate_normal(np.array(gMean)[0], np.array(gCov), allow_singular=True)
    
    #nDraws = int(popSize*size/totalPointsInGroups)
    nDraws = size
    if nDraws == 0:
        pass
    elif nDraws==1:
        #print(nDraws)
        personRowCont = pd.DataFrame(gDist.rvs(size=nDraws)).T
    else:
        #print(nDraws)
        personRowCont = pd.DataFrame(gDist.rvs(size=nDraws))
    if nDraws>0:
        personRowCont.columns=continuousVars
        #personRowCat = pd.concat([pd.DataFrame([gender, smoking, raceEthnicity, statin, 
        #                            education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount]).T]*nDraws,
        #                        ignore_index=True)
        personRowCat = pd.concat([pd.DataFrame(group).T]*nDraws, ignore_index=True)
        personRowCat.columns = categoricalVars
        personRow = pd.concat( [pd.Series(names), personRowCat, personRowCont], axis=1).rename(columns={0:"name"})
        personsDf = pd.concat([personsDf,personRow])
    

In [15]:
personsDf.head()

,name,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,41909,1,0,1,False,1,0,0,0.0,62.157082,45.814063,36.589655,226.430582,108.345160,6.263221,155.639129,123.571332,0.989469,122.375309,65.381554
1,42688,1,0,1,False,1,0,0,0.0,49.992311,44.946865,28.189541,153.619603,110.972330,5.859257,88.971266,102.506521,0.593414,125.949251,88.682632
2,43025,1,0,1,False,1,0,0,0.0,15.983673,39.375618,33.380314,234.115627,251.663072,5.772535,139.664001,109.788199,0.851678,111.377956,87.864916
3,43390,1,0,1,False,1,0,0,0.0,38.715531,39.940968,38.184832,185.337142,242.284426,5.553624,92.645844,127.502329,0.894123,123.191501,65.891548
4,44501,1,0,1,False,1,0,0,0.0,45.939015,40.457015,22.634844,160.320499,172.850585,5.664377,81.183563,85.895882,0.600087,145.303193,76.534771


In [16]:
personsDf.to_csv(dataDir+"/nhanes-persons-from-Gaussians.csv", index=False)